In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import json
import re
from time import sleep

In [ ]:
url = "http://house.gov.by/ru/deputies-ru/"
main_page = requests.get(url)
soup = bs(main_page.content, "html5lib")
links = [item.find("a")["href"] for item in soup.find_all("div", {"class": "news_item_second"})]

In [ ]:
data = {}
for link in links:
    page = requests.get(link).content
    soup = bs(page, "html5lib")
    fio = soup.find("h1").get_text()
    data[fio] = {}
    img_url = soup.find("div", {"class": "dep_img"}).find("img")["src"]
    img_name = re.match("^.+person/(.+)$", img_url).group(1)
    with open("/home/ushchent/vybory_2019/6_sozyv/img/{}".format(img_name), "wb") as f:
        image = requests.get(img_url)
        f.write(image.content)
    bio = soup.find("div", {"id": "biography_info"}).get_text()
    data[fio]["id"] = img_name
    data[fio]["bio"] = bio
    sleep(2)

In [ ]:
with open("../web_parsers/ppns/vi_sozyv_data/deputaty_vi.json", "w") as f:
    f.write(json.dumps(data, ensure_ascii=False))